In [17]:
#!pip install llama-index
#!pip install llama-index-embeddings-huggingface
#!pip install llama-index-embeddings-instructor
#!pip install llama-index-llms-ollama
#!pip install llama-index
#!pip install ollama
from llama_index.core import SimpleDirectoryReader, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings



import pickle
# image and video processing
import ollama
from ollama import generate
import glob
import pandas as pd
from PIL import Image
import os
from io import BytesIO

ModuleNotFoundError: No module named 'llama_index.data_transformer'

# image and video

In [12]:

input_dir_path="/Users/farshid/farshid/pirahansiah.github.io/src/LLMs/" #'/Users/farshid/farshid/pirahansiah.github.io/src/books/goal'
output_dir_path=input_dir_path+'/output'+'image_descriptions.csv'

In [13]:
def load_or_create_dataframe(filename):
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['image_file', 'description'])
    return df
def get_png_files(folder_path):
    return glob.glob(f"{folder_path}/*.jpg")
def process_image(image_file,df):
    print(f"\nProcessing {image_file}\n")
    with Image.open(image_file) as img:
        with BytesIO() as buffer:
            img.save(buffer, format='PNG')
            image_bytes = buffer.getvalue()
    full_response = ''
    # Generate a description of the image  llava:13b-v1.6
    for response in generate(model='llava:latest', 
                             prompt='describe this image and make sure to include anything notable about it \
                             (include text you see in the image) \
                               also relative location of main objects, \
                                also if include QR-code add the information of it into the output:', 
                             images=[image_bytes], 
                             stream=True):
        print(response['response'], end='', flush=True)
        full_response += response['response']
    df.loc[len(df)] = [image_file, full_response]


In [14]:
image_files = get_png_files("/Users/farshid/farshid/pirahansiah.github.io/src/LLMs/") #("./images") 
df = load_or_create_dataframe(output_dir_path)
image_files.sort()
print(image_files[:3])
print(df.head())
for image_file in image_files:
    if image_file not in df['image_file'].values:
        process_image(image_file,df)
df.to_csv('image_descriptions.csv', index=False)



['/Users/farshid/farshid/pirahansiah.github.io/src/LLMs/a.jpg']
Empty DataFrame
Columns: [image_file, description]
Index: []

Processing /Users/farshid/farshid/pirahansiah.github.io/src/LLMs/a.jpg

 The image displays a social media post featuring a QR code. The background of the post is white, and there's text at the top which appears to be in Persian (Farsi) script, although the content of the text is not clear due to its resolution.

In the foreground, centered on the image, is a smartphone with a QR code displayed on its screen. The phone has a dark background and is showing the QR code with a focus on the center. The smartphone is encased in a light brown frame that mimics the appearance of a smartphone case or stand.

In the top right corner, there's an avatar of a person, presumably the user who posted this image. Below the avatar, there's a line of text which seems to be the username or handle associated with the post. The text reads "Farshid Piranpahl Sahal".

The overall styl

# data
## files
### pdf, md, txt, csv, ...

In [ ]:
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)
Settings.embed_model = embed_model 
index = VectorStoreIndex.from_documents(docs)
with open('index_data.pkl', 'wb') as file:
    pickle.dump(index, file)

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)
loader = SimpleDirectoryReader(
    input_dir=input_dir_path,
    required_exts=[".pdf"],
    recursive=True
)
docs = loader.load_data()
index = VectorStoreIndex.from_documents(docs)


In [16]:
from llama_index.data_transformer import DocumentTransformer
def extract_metadata(file_path, file_content):
    metadata = {
        "file_name": file_path.name,
        "file_size": len(file_content),
    }
    return metadata
metadata_transformer = DocumentTransformer(extract_metadata=extract_metadata, exclude_source=True)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)
loader = SimpleDirectoryReader(
    input_dir=input_dir_path,
    required_exts=[".pdf"],
    recursive=True
)
docs = loader.load_data()
transformed_docs = metadata_transformer.transform_documents(docs)
index = VectorStoreIndex.from_documents(transformed_docs)


NameError: name 'DocumentTransformer' is not defined

In [19]:


def extract_metadata(file_path, file_content):
    metadata = {
        "file_name": file_path.name,
        "file_size": len(file_content),
    }
    return metadata

def add_metadata_to_documents(documents, metadata_extractor):
    for document in documents:
        metadata = metadata_extractor(document.file_path, document.content)
        document.metadata = metadata
    return documents

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

loader = SimpleDirectoryReader(
    input_dir=input_dir_path,
    required_exts=[".pdf"],
    recursive=True
)

docs = loader.load_data()
docs_with_metadata = add_metadata_to_documents(docs, extract_metadata)
index = VectorStoreIndex.from_documents(docs_with_metadata)

python(45782) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


AttributeError: 'Document' object has no attribute 'file_path'

In [30]:
# another 1
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings
import argparse
import os
import shutil
import pandas as pd
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
#from langchain.text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

DATA_PATH = input_dir_path #"data"
CSV_PATH = "chunks.csv"

def get_embedding_function():
    embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )
    # embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

In [44]:
import argparse
import os
import shutil
import pandas as pd
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

DATA_PATH = input_dir_path  # "data"
CSV_PATH = "chunks.csv"
RESET_CSV = True  # Set to True if you want to clear the CSV and start fresh

def main():
    if RESET_CSV:
        print("✨ Clearing CSV")
        clear_csv()

    documents = load_documents()
    chunks = split_documents(documents)
    add_to_csv(chunks)

def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

def add_to_csv(chunks: list[Document]):
    chunks_with_ids = calculate_chunk_ids(chunks)
    data = [{'id': chunk.metadata['id'],
             'content': chunk.page_content,  # Replace 'content' with 'page_content'
             'source': chunk.metadata.get('source'),
             'page': chunk.metadata.get('page')} for chunk in chunks_with_ids]
    df = pd.DataFrame(data)

    if RESET_CSV or not os.path.exists(CSV_PATH):
        df.to_csv(CSV_PATH, index=False)
    else:
        df.to_csv(CSV_PATH, mode='a', header=False, index=False)

def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id
        chunk.metadata["id"] = chunk_id

    return chunks

def clear_csv():
    if os.path.exists(CSV_PATH):
        os.remove(CSV_PATH)

# Run the main function
main()

✨ Clearing CSV


In [43]:
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

\---

Answer the question based on the above context: {question}
"""



def query_csv(query_text, csv_file):
    # Load the CSV file
    df = pd.read_csv(csv_file)

    # Search for relevant chunks
    relevant_chunks = df[df['content'].str.contains(query_text, case=False)]

    # Combine the relevant chunks into a single context
    context_text = "\n\n---\n\n".join(relevant_chunks['content'].tolist())

    if not context_text:
        return "No relevant information found in the CSV file."

    # Prepare the prompt
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # Generate the response - llama3:8b
    model = Ollama(model="phi3")
    response_text = model.invoke(prompt)

    # Format the response with sources
    sources = relevant_chunks['id'].tolist()
    formatted_response = f"Response: {response_text}\nSources: {sources}"

    return formatted_response

if __name__ == "__main__":
    query_text = input("Enter your query: ")
    csv_file = "chunks.csv"  # Replace with the actual path to your CSV file

    response = query_csv(query_text, csv_file)
    print(response)

No relevant information found in the CSV file.


In [39]:
query_text = input("Enter your query: ")
csv_file = "chunks.csv"  # Replace with the actual path to your CSV file

response = query_csv(query_text, csv_file)
print(response)

No relevant information found in the CSV file.


In [45]:
!llamafile -ngl 9999 \
  -m llava-v1.5-7b-mmproj-Q4_0.gguf \
  --mmproj llava-v1.5-7b-mmproj-Q8_0.gguf \
  --host 0.0.0.0

zsh:1: command not found: llamafile


python(76088) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
./llava-v1.5-7b-mmproj-Q4_0.llamafile -ngl 9999 --temp 0.2 --image ./a.jpg -e -p '### User: What do you see?\n### Assistant:'

./llamafile -m llava-v1.5-7b-mmproj-Q4_0.gguf -ngl 9999
./llamafile -m phi3 -ngl 9999

./llamafile -m llava-v1.5-7b-q4.llamafile -ngl 9999


./llava-v1.5-7b-q4.llamafile -ngl 9999 

--temp 0.2 --image ./a.jpg -e -p '### User: What do you see?\n### Assistant:'


In [48]:
#./llamafile -m phi3 -ngl 9999
#!/usr/bin/env python3
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="phi3",
    messages=[
        {"role": "system", "content": "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
        {"role": "user", "content": "python code to read a file"},
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Certainly! To read a file in Python, you can use the built-in `open()` function along with a `with` statement for proper resource management. Here\'s a simple example that reads the content of a file and prints it:\n\n```python\ndef read_file(file_path):\n    try:\n        with open(file_path, \'r\') as file:\n            content = file.read()\n            print(content)\n    except FileNotFoundError:\n        print(f"The file {file_path} was not found.")\n    except Exception as e:\n        print(f"An error occurred: {e}")\n\n# Replace \'yourfile.txt\' with the path to your file\nfile_path = \'yourfile.txt\'\nread_file(file_path)\n```\n\nThis function takes the file path as an argument, attempts to open and read the file, and prints its content. It also handles common errors like the file not being found. Remember to replace `\'yourfile.txt\'` with the actual path to your file.\n\nFor larger files or more complex operations, consider using a loop to read

In [53]:
    # if len(sys.argv) != 2:
    #     print("Usage: python script.py <image_path>")
    #     sys.exit(1)

    # image_path = sys.argv[1]


#./llava-v1.5-7b-q4.llamafile -ngl 9999 
import sys
from PIL import Image
import base64
import requests
def image_to_base64(image_path):
    try:
        with open(image_path, "rb") as image_file:
            image_data = image_file.read()
            base64_representation = base64.b64encode(image_data).decode("utf-8")
            return base64_representation
    except Exception as e:
        print(f"Error: {e}")
        return None
def main():
    image_path = "/Users/farshid/farshid/pirahansiah.github.io/src/LLMs/a.jpg"
    base64_image = image_to_base64(image_path)
    print(base64_image)
    if base64_image:
        payload =  {
        "stream":False,
        "image_data":[{"id":10,"data":base64_image}],
        "prompt":"Your are a AI assistant that describes images.\nUSER: What does the [img-10] contain?\nASSISTANT:"
         }
        headers = {"Content-Type": "application/json"}
        #make sure that Llamafile server is running on 8080  completion
        response = requests.post('http://localhost:8080/completion', headers=headers, json=payload)
        r = response.json()
        print(r["content"])
    else:
        print("There was an error !")
main()

...
 The image is a picture of a barcode, presumably from a product or document. The barcode has a photo attached to the top right of the frame, possibly of a person who is not visible in the frame. The picture is placed on a background that is most likely white, as the barcode itself is the primary focus of the image.
